In [4]:
import csv
from collections import namedtuple

anime_data = []
with open("data/processed/processed_animes.csv", encoding="utf8", newline="") as f:
    reader = csv.reader(f, delimiter=",", quotechar='"')
    fields = next(reader)
    AnimeInfo = namedtuple("AnimeInfo", fields)
    for item_id, title, synopsis, genre, score, tokenized_synopsis in reader:
        item_id = int(item_id)
        genre = eval(genre)
        score = float(score)
        tokenized_synopsis = eval(tokenized_synopsis)
        anime_data.append(AnimeInfo(item_id, title, synopsis, genre, score, tokenized_synopsis))

review_data = []
with open("data/processed/processed_reviews.csv", encoding="utf8", newline="") as f:
    reader = csv.reader(f, delimiter=",", quotechar='"')
    fields = next(reader)
    ReviewInfo = namedtuple("ReviewInfo", fields)
    for review_id, user_id, item_id, text, rating, tokenized_text in reader:
        review_id = int(review_id)
        user_id = int(user_id)
        item_id = int(item_id)
        rating = int(rating)
        tokenized_text = eval(tokenized_text)
        review_data.append(ReviewInfo(review_id, user_id, item_id - 1, text, rating, tokenized_text))

In [23]:
print("# of anime:", len(set(a.item_id for a in anime_data)))
print("# of users:", len(set(r.user_id for r in review_data)))

# of anime: 2258
# of users: 1529


In [24]:
user_to_review_idxs = {r.user_id : [] for r in review_data}
for i,r in enumerate(review_data):
    user_to_review_idxs[r.user_id].append(i)

In [25]:
# do hold-one-out for evaluation

import random

train_split = []
test_split = []

for _, reviews in user_to_review_idxs.items():
    hold_out = random.choice(reviews)
    test_split.append(hold_out)
    for ridx in reviews:
        if ridx != hold_out:
            train_split.append(ridx)

print(len(train_split), len(test_split))

34083 1529


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

In [13]:
class SmallNCF(nn.Module):
    def __init__(self, add_anime_embedding=False, add_user_embedding=False):
        super(SmallNCF, self).__init__()
        self.add_anime_embedding = add_anime_embedding
        self.add_user_embedding = add_user_embedding
        
        user_emb_dim = 500
        anime_emb_dim = 500
        
        if add_anime_embedding:
            self.anime_embedding = nn.Embedding(2258, anime_emb_dim)
        
        if add_user_embedding:
            self.user_embedding = nn.Embedding(1529, user_emb_dim)
        
        self.fc1 = nn.Linear(user_emb_dim + anime_emb_dim, 50)
        self.fc2 = nn.Linear(50, 1)
        self.relu = nn.LeakyReLU(0.05)
        self.dropout = nn.Dropout(0.0)
    
    def forward(self, user, anime):
        # second dim only present if not adding embeddings
        # user: (batch_size, [user_emb_dim])
        # anime: (batch_size, [anime_emb_dim])
        
        if self.add_anime_embedding:
            anime = self.anime_embedding(anime)
        
        if self.add_user_embedding:
            user = self.user_embedding(user)
        
        x = torch.cat((user, anime), dim=1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [14]:
class NCF(nn.Module):
    def __init__(self, add_anime_embedding=False, add_user_embedding=False):
        super(NCF, self).__init__()
        self.add_anime_embedding = add_anime_embedding
        self.add_user_embedding = add_user_embedding
        
        user_emb_dim = 768
        anime_emb_dim = 768
        
        if add_anime_embedding:
            self.anime_embedding = nn.Embedding(2258, anime_emb_dim)
        
        if add_user_embedding:
            self.user_embedding = nn.Embedding(1529, user_emb_dim)
        
        self.fc1 = nn.Linear(user_emb_dim + anime_emb_dim, 768)
        self.fc2 = nn.Linear(768, 384)
        self.fc3 = nn.Linear(384, 96)
        self.fc4 = nn.Linear(96, 1)
        self.relu = nn.LeakyReLU(0.05)
        self.dropout = nn.Dropout(0.3)
    
    def forward(self, user, anime):
        # second dim only present if not adding embeddings
        # user: (batch_size, [user_emb_dim])
        # anime: (batch_size, [anime_emb_dim])
        
        if self.add_anime_embedding:
            anime = self.anime_embedding(anime)
        
        if self.add_user_embedding:
            user = self.user_embedding(user)
        
        x = torch.cat((user, anime), dim=1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

In [28]:
## First try basic NCF

model = SmallNCF(True, True).cuda()
opt = optim.Adam(model.parameters(), lr=0.003)

for epoch in range(20):
    print(f"Epoch {epoch+1}")
    
    model.train()
    
    random.shuffle(train_split)
    
    total_epoch_loss = 0
    minibatch_count = 0
    
    for b_idx in range(0, 64*(len(train_split)//64), 64):
        opt.zero_grad()
        users = torch.LongTensor([review_data[train_split[i]].user_id for i in range(b_idx, b_idx + 64)]).cuda()
        anime = torch.LongTensor([review_data[train_split[i]].item_id for i in range(b_idx, b_idx + 64)]).cuda()
        ratings = torch.FloatTensor([review_data[train_split[i]].rating for i in range(b_idx, b_idx + 64)]).cuda()
        
        pred = model(users, anime)
        loss = ((ratings - pred)**2).mean()
        loss.backward()
        opt.step()
        
        total_epoch_loss += loss.item()
        minibatch_count += 1
    
    with torch.no_grad():
        model.eval()
        test_users = torch.LongTensor([review_data[idx].user_id for idx in test_split]).cuda()
        test_anime = torch.LongTensor([review_data[idx].item_id for idx in test_split]).cuda()
        test_ratings = torch.FloatTensor([review_data[idx].rating for idx in test_split]).cuda()
        pred = model(test_users, test_anime)
        test_loss = ((test_ratings - pred) ** 2).mean().item()
        
    print(f"train loss {total_epoch_loss/minibatch_count:.2f}")
    print(f"test loss {test_loss:.2f}")

Epoch 1
train loss 7.81
test loss 5.38
Epoch 2
train loss 5.20
test loss 5.24
Epoch 3
train loss 5.07
test loss 5.52
Epoch 4
train loss 5.03
test loss 5.08
Epoch 5
train loss 5.02
test loss 5.07
Epoch 6
train loss 4.99
test loss 5.06
Epoch 7
train loss 4.99
test loss 5.29
Epoch 8
train loss 4.95
test loss 5.04
Epoch 9
train loss 4.93
test loss 5.12
Epoch 10
train loss 4.94
test loss 5.07
Epoch 11
train loss 4.92
test loss 4.96
Epoch 12
train loss 4.94
test loss 5.02
Epoch 13
train loss 4.92
test loss 5.11
Epoch 14
train loss 4.89
test loss 4.93
Epoch 15
train loss 4.91
test loss 5.00
Epoch 16
train loss 4.89
test loss 4.94
Epoch 17
train loss 4.88
test loss 4.93
Epoch 18
train loss 4.91
test loss 4.99
Epoch 19
train loss 4.88
test loss 4.88
Epoch 20
train loss 4.88
test loss 5.03


In [26]:
user_totals = {r.user_id : [0, 0] for r in review_data}

for i in train_split:
    user_totals[review_data[i].user_id][0] += review_data[i].rating
    user_totals[review_data[i].user_id][1] += 1

user_avgs = {k : (a / b) for k,(a,b) in user_totals.items()}

r = 0

for j in test_split:
    pred = user_avgs[review_data[j].user_id]
    r += (pred - review_data[j].rating)**2

print(r / len(test_split))

3.713802968624789


In [27]:
anime_avgs = {a.item_id : a.score for a in anime_data}

r = 0
for j in test_split:
    pred = anime_avgs[review_data[j].item_id]
    r += (pred - review_data[j].rating)**2

print(r / len(test_split))

4.242937017658601
